In [15]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
import plotly.express as px

In [16]:
spark = (
    SparkSession.builder.appName("iot").getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
spark.sparkContext.version


'4.0.0'

Credits to Anton T. Ruberts:
https://www.youtube.com/watch?v=2LG2hUQxLmA

Dataset from
https://www.kaggle.com/datasets/agungpambudi/network-malware-detection-connection-analysis?resource=download

In [17]:
file_path = r"C:\Users\gabyl\OneDrive\Desktop\Proyectos\pyspark\data" + "\IoT-Malware.csv"
df = spark.read.option('delimiter', '|').csv(file_path, inferSchema=True, header=True)
df.show(5)

<>:1: SyntaxWarning: invalid escape sequence '\I'
<>:1: SyntaxWarning: invalid escape sequence '\I'
C:\Users\gabyl\AppData\Local\Temp\ipykernel_5028\1134689689.py:1: SyntaxWarning: invalid escape sequence '\I'
  file_path = r"C:\Users\gabyl\OneDrive\Desktop\Proyectos\pyspark\data" + "\IoT-Malware.csv"


+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|                 ts|               uid|      id.orig_h|id.orig_p|      id.resp_h|id.resp_p|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|    label|      detailed-label|
+-------------------+------------------+---------------+---------+---------------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+---------+--------------------+
|1.525879831015811E9|CUmrqr4svHuSXJy5z7|192.168.100.103|  51524.0| 65.127.233.163|     23.0|  tcp|      -|2.999051|         0|         0|     

In [18]:
df.printSchema()

root
 |-- ts: double (nullable = true)
 |-- uid: string (nullable = true)
 |-- id.orig_h: string (nullable = true)
 |-- id.orig_p: double (nullable = true)
 |-- id.resp_h: string (nullable = true)
 |-- id.resp_p: double (nullable = true)
 |-- proto: string (nullable = true)
 |-- service: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- orig_bytes: string (nullable = true)
 |-- resp_bytes: string (nullable = true)
 |-- conn_state: string (nullable = true)
 |-- local_orig: string (nullable = true)
 |-- local_resp: string (nullable = true)
 |-- missed_bytes: double (nullable = true)
 |-- history: string (nullable = true)
 |-- orig_pkts: double (nullable = true)
 |-- orig_ip_bytes: double (nullable = true)
 |-- resp_pkts: double (nullable = true)
 |-- resp_ip_bytes: double (nullable = true)
 |-- tunnel_parents: string (nullable = true)
 |-- label: string (nullable = true)
 |-- detailed-label: string (nullable = true)



### Pre-processing

In [19]:
df = df.withColumn("dt", F.from_unixtime("ts")).withColumn("dt", F.to_timestamp("dt"))

In [20]:
df.select("dt").show(3)

+-------------------+
|                 dt|
+-------------------+
|2018-05-09 17:30:31|
|2018-05-09 17:30:31|
|2018-05-09 17:30:31|
+-------------------+
only showing top 3 rows


In [21]:
df = df.withColumnsRenamed(
    {
        "id.orig_h": "source_ip",
        "id.orig_p": "source_port",
        "id.resp_h": "dest_ip",
        "id.resp_p": "dest_port",
    }
)

In [22]:
df.columns

['ts',
 'uid',
 'source_ip',
 'source_port',
 'dest_ip',
 'dest_port',
 'proto',
 'service',
 'duration',
 'orig_bytes',
 'resp_bytes',
 'conn_state',
 'local_orig',
 'local_resp',
 'missed_bytes',
 'history',
 'orig_pkts',
 'orig_ip_bytes',
 'resp_pkts',
 'resp_ip_bytes',
 'tunnel_parents',
 'label',
 'detailed-label',
 'dt']

### Dataset quality checks

In [23]:
df.agg(
    F.min("dt").alias("min_date"),
    F.max("dt").alias("max_date")
).show()

+-------------------+-------------------+
|           min_date|           max_date|
+-------------------+-------------------+
|2018-05-09 17:30:31|2018-05-14 09:24:43|
+-------------------+-------------------+



In [24]:
# df shape:
df.count(), len(df.columns)

(1008748, 24)

In [25]:
to_analyse = [
    "source_ip",
    "source_port",
    "dest_ip",
    "dest_port",
    "proto",
    "service",
    "duration",
    "orig_bytes",
    "resp_bytes",
    "conn_state",
    "local_orig",
    "local_resp",
    "missed_bytes",
    "history",
    "orig_pkts",
    "orig_ip_bytes",
    "resp_pkts",
    "resp_ip_bytes",
    "tunnel_parents",
    "label",
    "detailed-label",
]

unique_counts = df.agg(
        *(F.countDistinct(
            F.col(c)
            ).alias(c) for c in to_analyse))
print(unique_counts.show())

+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+-----+--------------+
|source_ip|source_port|dest_ip|dest_port|proto|service|duration|orig_bytes|resp_bytes|conn_state|local_orig|local_resp|missed_bytes|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|tunnel_parents|label|detailed-label|
+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+----------+----------+------------+-------+---------+-------------+---------+-------------+--------------+-----+--------------+
|    15004|      28243| 597107|    65426|    3|      5|   16650|       171|       479|        11|         1|         1|           1|    126|       54|         1249|       69|         1141|             1|    2|             3|
+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+---

In [26]:
# unique_counts = unique_counts.first()
static_cols = [c for c in unique_counts.asDict() if unique_counts[c] == 1]
print(static_cols)
df = df.drop(*static_cols)

PySparkAttributeError: [ATTRIBUTE_NOT_SUPPORTED] Attribute `asDict` is not supported.

### Count Nulls

In [ ]:
df = df.replace("-", None)
non_static_cols = [c for c in df.columns if c not in static_cols]
df.select(
    [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in non_static_cols]
).show()

+---+---+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+-------+---------+-------------+---------+-------------+-----+--------------+---+
| ts|uid|source_ip|source_port|dest_ip|dest_port|proto|service|duration|orig_bytes|resp_bytes|conn_state|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|label|detailed-label| dt|
+---+---+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+-------+---------+-------------+---------+-------------+-----+--------------+---+
|  0|  0|        0|          0|      0|        0|    0|1005507|  796300|    796300|    796300|         0|  17421|        0|            0|        0|            0|    0|        469275|  0|
+---+---+---------+-----------+-------+---------+-----+-------+--------+----------+----------+----------+-------+---------+-------------+---------+-------------+-----+--------------+---+



### Time Series

In [ ]:
df = df.withColumns(
    {
        "day": F.date_trunc("day", "dt"),
        "hour": F.date_trunc("hour", "dt"),
        "minute": F.date_trunc("minute", "dt"),
    }
)

In [ ]:
df.groupBy(["hour", "label"]).agg(F.count("uid").alias('counts')).orderBy("hour").toPandas()

,hour,label,counts
0,2018-05-09 17:00:00,Benign,2197
1,2018-05-09 17:00:00,Malicious,2623
2,2018-05-09 18:00:00,Malicious,5420
3,2018-05-09 18:00:00,Benign,4346
4,2018-05-09 19:00:00,Malicious,5402
...,...,...,...
221,2018-05-14 07:00:00,Malicious,4399
222,2018-05-14 08:00:00,Benign,4036
223,2018-05-14 08:00:00,Malicious,4325
224,2018-05-14 09:00:00,Benign,1608


### Univariate Data Analysis

In [ ]:
inter_df = df.groupBy("proto").count().alias("count").orderBy("count", ascending=True)
inter_df = inter_df.withColumn(
    "percent", F.round(F.col("count")/F.sum(F.col("count")).over(Window.partitionBy()), 4)
    )
inter_df.show()

+-----+------+-------+
|proto| count|percent|
+-----+------+-------+
| icmp| 17421| 0.0173|
|  udp|408193| 0.4047|
|  tcp|583134| 0.5781|
+-----+------+-------+



In [27]:
def counts(df, var):
    var_counts = df.groupBy(var).count().orderBy("count", ascending=False)
    var_counts = var_counts.withColumn(
        "percent", F.round(F.col("count")/F.sum(F.col("count")).over(Window.partitionBy()), 4)
    )
    var_counts.show()
    fig = px.bar(var_counts.toPandas(), x=var, y="count")
    fig.show()

categorical_cols = ["proto", "service", "conn_state", "history", "label"]
for col in categorical_cols:
    counts(df,col)

+-----+------+-------+
|proto| count|percent|
+-----+------+-------+
|  tcp|583134| 0.5781|
|  udp|408193| 0.4047|
| icmp| 17421| 0.0173|
+-----+------+-------+



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

## Prepare for Modelling

## Full Pipeline

In [ ]:
static_cols = ["local_orig", "local_resp", "missed_bytes", "tunnel_parents"]
recast_cols = {
    "duration": F.col("duration").cast("double"),
    "orig_bytes": F.col("orig_bytes").cast("double"),
    "resp_bytes": F.col("resp_bytes").cast("double"),
    "orig_ip_bytes": F.col("orig_ip_bytes").cast("double"),
    "orig_pkts": F.col("orig_pkts").cast("double"),
    "resp_pkts": F.col("resp_pkts").cast("double"),
    "resp_ip_bytes": F.col("resp_ip_bytes").cast("double"),
}

fill_vals = {
    "duration": -999999,
    "orig_bytes": -999999,
    "resp_bytes": -999999,
    "orig_pkts": -999999,
    "orig_ip_bytes": -999999,
    "resp_pkts": -999999,
    "resp_ip_bytes": -999999,
    "history": "missing",
    "proto": "missing",
    "service": "missing",
    "conn_state": "missing",
}

preprocessed_data = (
    spark.read.option("delimiter", '|')
    .csv(file_path, inferSchema=True, header=True)
    .withColumn("dt", F.to_timestamp(F.from_unixtime("ts")))
    .withColumns(
        {
            "day": F.date_trunc("day","dt"),
            "hour": F.date_trunc("hour","dt"),
            "minute": F.date_trunc("minute","dt"),
            "second": F.date_trunc("second","dt"),
        }
    )
    .withColumnsRenamed(
        {
            "id.orig_h": "source_ip",
            "id.orig_p": "source_port",
            "id.resp_h": "dest_ip",
            "id.resp_p": "dest_port"
        }
    )
    .drop(*static_cols)
    .replace("-", None)
    .withColumns(recast_cols)
    .fillna(fill_vals)
)

preprocessed_data.show()


+-------------------+------------------+---------------+-----------+---------------+---------+-----+-------+---------+----------+----------+----------+-------+---------+-------------+---------+-------------+---------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+
|                 ts|               uid|      source_ip|source_port|        dest_ip|dest_port|proto|service| duration|orig_bytes|resp_bytes|conn_state|history|orig_pkts|orig_ip_bytes|resp_pkts|resp_ip_bytes|    label|      detailed-label|                 dt|                day|               hour|             minute|             second|
+-------------------+------------------+---------------+-----------+---------------+---------+-----+-------+---------+----------+----------+----------+-------+---------+-------------+---------+-------------+---------+--------------------+-------------------+-------------------+-------------------+-------------------+----

## Write Out

In [ ]:
# Note the forward slashes and the file:/// prefix
output_dir = r'file:///C:/Users/gabyl/OneDrive/Desktop/Proyectos/pyspark/output/preprocessing'

# Your write command remains the same
df.write.parquet(output_dir + "/df.parquet", mode="overwrite")

Py4JJavaError: An error occurred while calling o2193.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:817)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1415)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1620)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:739)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2078)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2122)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:961)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2078)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2122)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(F

In [ ]:
read_in = spark.read.parquet("processed.pq")
read_in.show()